### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

Мягкий дедлайн 28 Сентября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 5 Октября (Итоговая проверка)

In [5]:
import implicit
import pandas as pd
import time
import numpy as np
import scipy.sparse as sp
from copy import deepcopy as dp
from tqdm import tqdm
from lightfm.datasets import fetch_movielens

/Users/sergekulesh/opt/miniconda3/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [6]:
ratings = pd.read_csv('ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [7]:
movie_info = pd.read_csv('movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [8]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [9]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [10]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [11]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [12]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [13]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [14]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [16]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '360    Lion King, The (1994)',
 '1838    Mulan (1998)',
 '1526    Hercules (1997)',
 '2618    Tarzan (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [17]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [18]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [19]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [20]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '2502    Matrix, The (1999)',
 '1182    Aliens (1986)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '3402    Close Encounters of the Third Kind (1977)',
 '847    Godfather, The (1972)',
 '2460    Planet of the Apes (1968)',
 '2880    Dr. No (1962)']

ratings.movie_idТеперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [17]:
class SVD:
    def __init__(self, learning_rate = 1e-3, lambda_l2 = 1e-3, 
                 epsilon = 1e-3, latent_size = 16, max_iterations = int(1e7), verbose = int(1e5)):
        self.learning_rate = learning_rate
        self.lambda_l2 = lambda_l2
        self.epsilon = epsilon
        self.latent_size = latent_size
        self.max_iterations = max_iterations
        self.verbose = verbose 
        
    def fit(self, ratings, movie_info):
        # функция поиска описания фильма по его идентификатору
        self.search = lambda x: movie_info.loc[movie_info.movie_id == x]
        
        # инициализация матриц эмбедингов
        self.W = np.random.uniform(0, 1/np.sqrt(self.latent_size), 
                      size=(ratings.user_id.max(), self.latent_size)) 
        
        self.H = np.random.uniform(0, 1/np.sqrt(self.latent_size), 
                      size=(self.latent_size, ratings.movie_id.max())) 
        
        # инициализация векторов смещения
        self.Wb = np.random.normal(2.5, 1, ratings.user_id.max()).reshape(-1, 1) 
        self.Hb = np.random.normal(2.5, 1, ratings.movie_id.max()).reshape(1, -1) 
        self.mu = np.mean(ratings.rating.mean())
        
        ratings_numpy = ratings.to_numpy()
        self.movie_info = movie_info
        self.ratings = ratings
    
        l = ratings.shape[0]
        itx = 0
        
        while True:
            
            if not itx % self.verbose:
                # подсчет rmse на кратной итерации
                self.PR = np.array(self.W @ self.H + self.Wb + self.Hb + self.mu)
                idx_users = ratings_numpy[:, 0] - 1
                idx_movies = ratings_numpy[:, 1] - 1
                rvals = ratings_numpy[:, 2]
                f_norm =  np.linalg.norm(self.PR[idx_users, idx_movies] - rvals) / l
                print(f'|{itx}. rmse - {f_norm:.5f}')
                if f_norm <= self.epsilon:
                    break
                
            # градиентый спуск по случайному известному элементу матрицы рейтингов
            i, j, r = ratings_numpy[np.random.randint(l)]
            i -= 1
            j -= 1
            
            err = self.W[i, :] @ self.H[:, j] + self.Wb[i, :] + self.Hb[:, j] + self.mu - r
            
            W_upd = self.W[i, :] - self.learning_rate * (err * self.H[:, j].T + self.lambda_l2 * self.W[i, :])
            H_upd = self.H[:, j] - self.learning_rate * (err * self.W[i, :].T + self.lambda_l2 * self.H[:, j])
            
            
            self.Wb[i, :] -= self.learning_rate * (err + self.lambda_l2 * self.Wb[i, :])
            self.Hb[:, j] -= self.learning_rate * (err + self.lambda_l2 * self.Hb[:, j])
            
            self.W[i, :] = W_upd
            self.H[:, j] = H_upd
            
            itx += 1
            if itx >= self.max_iterations:
                break
        
        self.movie_values = self.ratings.movie_id.sort_values().unique()
        print(f'Done! ({itx}, {f_norm:.5f})')
        
    def history(self, idx, count = 5):
        # выгрузка истории просмотренных фильмов по идентификатору пользователя
        idies = self.ratings.loc[self.ratings.user_id == idx].sort_values('rating').movie_id.iloc[:count].values
        histories = pd.concat([self.search(i) for i in idies])
        return histories.to_numpy()
        

    def similar_items(self, idx, count = 5):
        # предсказание схожих по представлениям фильмов
        similar_key = lambda x: np.linalg.norm(self.H[:, x] - self.H[:, idx - 1])
        idx_movies_real = self.ratings.movie_id.sort_values().unique() 
        distances = [[x, np.linalg.norm(similar_key(x - 1))] for x in idx_movies_real]
        distances.sort(key = lambda x: x[1])
        idies = [i[0] for i in distances[:count + 1]]
        similaries = pd.concat([self.search(i) for i in idies])
        return similaries.to_numpy()
    
    def recommend(self, idx, count = 5):
        # рекомендации пользователю 
        user_ratings_all = np.array(self.W @ self.H + self.Wb + self.Hb + self.mu)[idx - 1]
        user_ratings = set(ratings.loc[ratings.user_id == idx].movie_id)
        sorted_all_idies = np.argsort(user_ratings_all)[::-1] + 1
        idies = [i for i in sorted_all_idies if i not in user_ratings][:count]
        recomendations = pd.concat([self.search(i) for i in idies])
        return recomendations.to_numpy()

In [18]:
SVD_SGD = SVD(learning_rate = 1e-2, lambda_l2 = 0.1, latent_size = 64, epsilon = 1e-4, max_iterations = int(4e6), verbose = int(2e5))
SVD_SGD.fit(ratings, movie_info)

|0. rmse - 0.00564
|200000. rmse - 0.00193
|400000. rmse - 0.00145
|600000. rmse - 0.00125
|800000. rmse - 0.00114
|1000000. rmse - 0.00108
|1200000. rmse - 0.00104
|1400000. rmse - 0.00102
|1600000. rmse - 0.00100
|1800000. rmse - 0.00098
|2000000. rmse - 0.00097
|2200000. rmse - 0.00096
|2400000. rmse - 0.00095
|2600000. rmse - 0.00095
|2800000. rmse - 0.00094
|3000000. rmse - 0.00094
|3200000. rmse - 0.00094
|3400000. rmse - 0.00093
|3600000. rmse - 0.00093
|3800000. rmse - 0.00093
Done! (4000000, 0.00093)


In [22]:
SVD_SGD.similar_items(1)

array([[1, 'Toy Story (1995)', "Animation|Children's|Comedy"],
       [3114, 'Toy Story 2 (1999)', "Animation|Children's|Comedy"],
       [1196, 'Star Wars: Episode V - The Empire Strikes Back (1980)',
        'Action|Adventure|Drama|Sci-Fi|War'],
       [589, 'Terminator 2: Judgment Day (1991)',
        'Action|Sci-Fi|Thriller'],
       [1193, "One Flew Over the Cuckoo's Nest (1975)", 'Drama'],
       [34, 'Babe (1995)', "Children's|Comedy|Drama"]], dtype=object)

In [23]:
SVD_SGD.recommend(4, 10)

array([[878, 'Bye-Bye (1995)', 'Drama'],
       [2270, 'Century of Cinema, A (1994)', 'Documentary'],
       [285, 'Beyond Bedlam (1993)', 'Drama|Horror'],
       [3779, 'Project Moon Base (1953)', 'Sci-Fi'],
       [1360,
        'Identification of a Woman (Identificazione di una donna) (1982)',
        'Drama'],
       [2954, 'Penitentiary (1979)', 'Drama'],
       [1519, 'Broken English (1996)', 'Drama']], dtype=object)

In [24]:
SVD_SGD.history(4, 10)

array([[3527, 'Predator (1987)', 'Action|Sci-Fi|Thriller'],
       [1196, 'Star Wars: Episode V - The Empire Strikes Back (1980)',
        'Action|Adventure|Drama|Sci-Fi|War'],
       [1210, 'Star Wars: Episode VI - Return of the Jedi (1983)',
        'Action|Adventure|Romance|Sci-Fi|War'],
       [2951, 'Fistful of Dollars, A (1964)', 'Action|Western'],
       [1214, 'Alien (1979)', 'Action|Horror|Sci-Fi|Thriller'],
       [1036, 'Die Hard (1988)', 'Action|Thriller'],
       [480, 'Jurassic Park (1993)', 'Action|Adventure|Sci-Fi'],
       [1097, 'E.T. the Extra-Terrestrial (1982)',
        "Children's|Drama|Fantasy|Sci-Fi"],
       [3418, 'Thelma & Louise (1991)', 'Action|Drama'],
       [3702, 'Mad Max (1979)', 'Action|Sci-Fi']], dtype=object)

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [25]:
class ALS:
    def __init__(self, learning_rate = 1e-3, lambda_l2 = 1e-3, 
                 epsilon = 1e-3, latent_size = 16, max_iterations = int(1e7), verbose = int(1e5)):
        self.learning_rate = learning_rate
        self.lambda_l2 = lambda_l2
        self.epsilon = epsilon
        self.latent_size = latent_size
        self.max_iterations = max_iterations
        self.verbose = verbose 
        
    def fit(self, ratings, movie_info):
        self.search = lambda x: movie_info.loc[movie_info.movie_id == x]
        
        self.W = np.random.uniform(0, 1/np.sqrt(self.latent_size), 
                      size=(ratings.user_id.max(), self.latent_size)) 
        
        self.H = np.random.uniform(0, 1/np.sqrt(self.latent_size), 
                      size=(self.latent_size, ratings.movie_id.max()))     
        
        ratings_numpy = ratings.to_numpy()
        
        u_id = ratings_numpy[:, 0] - 1
        m_id = ratings_numpy[:, 1] - 1
        r_vl = ratings_numpy[:, 2]
        
        self.movie_info = movie_info
        self.ratings = ratings
    
        l = ratings.shape[0]
        itx = 0
        
        while True:
            self.PR = np.array(self.W @ self.H)
            if not itx % self.verbose:
                idx_users = ratings_numpy[:, 0] - 1
                idx_movies = ratings_numpy[:, 1] - 1
                rvals = ratings_numpy[:, 2]
                f_norm =  np.linalg.norm(self.PR[idx_users, idx_movies] - rvals) / l
                print(f'|{itx}. rmse - {f_norm:.5f}')
                if f_norm <= self.epsilon:
                    break
                    
            err = dp(self.PR)
            err[u_id, m_id] -= r_vl
            
            if not itx % 2:
                self.W -= self.learning_rate * (err @ self.H.T + self.lambda_l2 * self.W)
            else:
                self.H -= self.learning_rate * (self.W.T @ err + self.lambda_l2 * self.H)
            
            itx += 1
            if itx >= self.max_iterations:
                break
        
        self.movie_values = self.ratings.movie_id.sort_values().unique()
        print(f'Done! ({itx}, {f_norm:.5f})')
        
    def history(self, idx, count = 5):
        # выгрузка истории просмотренных фильмов по идентификатору пользователя
        idies = self.ratings.loc[self.ratings.user_id == idx].sort_values('rating').movie_id.iloc[:count].values
        histories = pd.concat([self.search(i) for i in idies])
        return histories.to_numpy()
        

    def similar_items(self, idx, count = 5):
        # предсказание схожих по представлениям фильмов
        similar_key = lambda x: np.linalg.norm(self.H[:, x] - self.H[:, idx - 1])
        idx_movies_real = self.ratings.movie_id.sort_values().unique() 
        distances = [[x, np.linalg.norm(similar_key(x - 1))] for x in idx_movies_real]
        distances.sort(key = lambda x: x[1])
        idies = [i[0] for i in distances[:count + 1]]
        similaries = pd.concat([self.search(i) for i in idies])
        return similaries.to_numpy()
    
    def recommend(self, idx, count = 5):
        # рекомендации пользователю 
        user_ratings_all = np.array(self.W @ self.H)[idx - 1]
        user_ratings = set(ratings.loc[ratings.user_id == idx].movie_id)
        sorted_all_idies = np.argsort(user_ratings_all)[::-1] + 1
        idies = [i for i in sorted_all_idies if i not in user_ratings][:count]
        recomendations = pd.concat([self.search(i) for i in idies])
        return recomendations.to_numpy()

In [26]:
ALS_SGD = ALS(learning_rate = 1e-3, lambda_l2 = 1e-3, latent_size = 64, epsilon = 1e-4, max_iterations = 150, verbose = 25)
ALS_SGD.fit(implicit_ratings, movie_info)

|0. rmse - 0.00550
|25. rmse - 0.00439
|50. rmse - 0.00395
|75. rmse - 0.00374
|100. rmse - 0.00370
|125. rmse - 0.00369
Done! (150, 0.00369)


In [27]:
ALS_SGD.similar_items(1)

array([[1, 'Toy Story (1995)', "Animation|Children's|Comedy"],
       [3114, 'Toy Story 2 (1999)', "Animation|Children's|Comedy"],
       [2355, "Bug's Life, A (1998)", "Animation|Children's|Comedy"],
       [588, 'Aladdin (1992)', "Animation|Children's|Comedy|Musical"],
       [364, 'Lion King, The (1994)', "Animation|Children's|Musical"],
       [2321, 'Pleasantville (1998)', 'Comedy']], dtype=object)

In [32]:
ALS_SGD.recommend(4, 20)

array([[1291, 'Indiana Jones and the Last Crusade (1989)',
        'Action|Adventure'],
       [1200, 'Aliens (1986)', 'Action|Sci-Fi|Thriller|War'],
       [1304, 'Butch Cassidy and the Sundance Kid (1969)',
        'Action|Comedy|Western'],
       [457, 'Fugitive, The (1993)', 'Action|Thriller'],
       [589, 'Terminator 2: Judgment Day (1991)',
        'Action|Sci-Fi|Thriller'],
       [2529, 'Planet of the Apes (1968)', 'Action|Sci-Fi'],
       [2571, 'Matrix, The (1999)', 'Action|Sci-Fi|Thriller'],
       [3471, 'Close Encounters of the Third Kind (1977)',
        'Drama|Sci-Fi'],
       [858, 'Godfather, The (1972)', 'Action|Crime|Drama'],
       [110, 'Braveheart (1995)', 'Action|Drama|War'],
       [1197, 'Princess Bride, The (1987)',
        'Action|Adventure|Comedy|Romance'],
       [2000, 'Lethal Weapon (1987)', 'Action|Comedy|Crime|Drama'],
       [969, 'African Queen, The (1951)', 'Action|Adventure|Romance|War'],
       [1953, 'French Connection, The (1971)',
        'Acti

In [33]:
ALS_SGD.history(4, 20)

array([[2951, 'Fistful of Dollars, A (1964)', 'Action|Western'],
       [1214, 'Alien (1979)', 'Action|Horror|Sci-Fi|Thriller'],
       [1036, 'Die Hard (1988)', 'Action|Thriller'],
       [480, 'Jurassic Park (1993)', 'Action|Adventure|Sci-Fi'],
       [2366, 'King Kong (1933)', 'Action|Adventure|Horror'],
       [1097, 'E.T. the Extra-Terrestrial (1982)',
        "Children's|Drama|Fantasy|Sci-Fi"],
       [3418, 'Thelma & Louise (1991)', 'Action|Drama'],
       [3702, 'Mad Max (1979)', 'Action|Sci-Fi'],
       [3468, 'Hustler, The (1961)', 'Drama'],
       [2692, 'Run Lola Run (Lola rennt) (1998)', 'Action|Crime|Romance'],
       [1201, 'Good, The Bad and The Ugly, The (1966)', 'Action|Western'],
       [1387, 'Jaws (1975)', 'Action|Horror'],
       [1954, 'Rocky (1976)', 'Action|Drama'],
       [1198, 'Raiders of the Lost Ark (1981)', 'Action|Adventure'],
       [2028, 'Saving Private Ryan (1998)', 'Action|Drama|War'],
       [260, 'Star Wars: Episode IV - A New Hope (1977)',
      

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [23]:
class BPR:
    def __init__(self, learning_rate = 1e-3, lambda_l2 = 1e-3, 
                 epsilon = 1e-3, latent_size = 16, max_iterations = int(1e7), verbose = int(1e5)):
        self.learning_rate = learning_rate
        self.lambda_l2 = lambda_l2
        self.epsilon = epsilon
        self.latent_size = latent_size
        self.max_iterations = max_iterations
        self.verbose = verbose 
        
    def fit(self, ratings, movie_info):
        
        
        # функция поиска описания фильма по его идентификатору
        self.search = lambda x: movie_info.loc[movie_info.movie_id == x]
        
        # инициализация матриц эмбедингов
        self.W = np.random.uniform(0, 1/np.sqrt(self.latent_size), 
                      size=(ratings.user_id.max(), self.latent_size)) 
        
        self.H = np.random.uniform(0, 1/np.sqrt(self.latent_size), 
                      size=(self.latent_size, ratings.movie_id.max())) 
        set_movies_all = set(np.arange(1, ratings.movie_id.max() + 1))
#         self.biasV = np.random.rand(len(set_movies_all)) * 0.01
        
        ratings_numpy = ratings.to_numpy()
        self.movie_info = movie_info
        self.ratings = ratings
    
        l = ratings.shape[0]
        itx = 0
        # множество всех фильмов
        
        while True:

            # подсчет rmse на кратной итерации
            self.PR = np.array(self.W @ self.H)
            idx_users = ratings_numpy[:, 0] - 1
            idx_movies = ratings_numpy[:, 1] - 1
            rvals = ratings_numpy[:, 2]
            f_norm =  np.linalg.norm(self.PR[idx_users, idx_movies] - rvals) / l
            if f_norm <= self.epsilon:
                break
                    
            for u_pos in tqdm(ratings.user_id.unique(), position=0, leave=False, desc=f'|{itx}. rmse - {f_norm:.10f}'):
                    u_movies = ratings.loc[ratings.user_id == u_pos].movie_id
                    m_pos = u_movies.sample().item()

                    u_pos -= 1
                    m_pos -= 1
                    # выделяем негативные случаи и берем случайный элемент
                    set_movies_pos = set(u_movies)
                    idies_neg = list(set_movies_all - set_movies_pos)
                    for _ in range(3):
                        m_neg = np.random.choice(idies_neg) - 1

                        # считаем расстояние от позитивного отзыва до негативного
                        r_pos = self.W[u_pos, :] @ self.H[:, m_pos]# + self.biasV[m_pos]
                        r_neg = self.W[u_pos, :] @ self.H[:, m_neg]# + self.biasV[m_neg]
                        err = r_pos - r_neg

                        loss = -1.0 / (1 + np.exp(err))
                        self.W[u_pos, :] -= self.learning_rate * (loss * (self.H[:, m_pos].T - self.H[:, m_neg].T)
                                                              + self.lambda_l2 * self.W[u_pos, :])

                        self.H[:, m_pos] -= self.learning_rate * (loss * self.W[u_pos, :].T + self.lambda_l2 * self.H[:, m_pos])
                        self.H[:, m_neg] -= self.learning_rate * (loss * self.W[u_pos, :].T + self.lambda_l2 * self.H[:, m_neg])

#                         self.biasV[m_pos] -= self.learning_rate * (loss + self.lambda_l2 * self.biasV[m_pos])
#                         self.biasV[m_neg] -= self.learning_rate * (-loss + self.lambda_l2 * self.biasV[m_neg])
                        
            itx += 1
            if itx >= self.max_iterations:
                break
        
        self.movie_values = self.ratings.movie_id.sort_values().unique()
        print(f'Done! ({itx}, {f_norm:.5f})')
        
    def history(self, idx, count = 5):
        # выгрузка истории просмотренных фильмов по идентификатору пользователя
        idies = self.ratings.loc[self.ratings.user_id == idx].sort_values('rating').movie_id.iloc[:count].values
        histories = pd.concat([self.search(i) for i in idies])
        return histories.to_numpy()
        

    def similar_items(self, idx, count = 5):
        # предсказание схожих по представлениям фильмов
        similar_key = lambda x: np.linalg.norm(self.H[:, x] - self.H[:, idx - 1])
        idx_movies_real = self.ratings.movie_id.sort_values().unique() 
        distances = [[x, np.linalg.norm(similar_key(x - 1))] for x in idx_movies_real]
        distances.sort(key = lambda x: x[1])
        idies = [i[0] for i in distances[:count + 1]]
        similaries = pd.concat([self.search(i) for i in idies])
        return similaries.to_numpy()
    
    def recommend(self, idx, count = 5):
        # рекомендации пользователю 
        user_ratings_all = np.array(self.W @ self.H)[idx - 1]
        user_ratings = set(ratings.loc[ratings.user_id == idx].movie_id)
        sorted_all_idies = np.argsort(user_ratings_all)[::-1] + 1
        idies = [i for i in sorted_all_idies if i not in user_ratings][:count]
        recomendations = pd.concat([self.search(i) for i in idies])
        return recomendations.to_numpy()

In [24]:
BPR_SGD = BPR(learning_rate = 5e-2, lambda_l2 = 1e-3, latent_size = 32, epsilon = 1e-4, max_iterations = 100, verbose = 1)
BPR_SGD.fit(implicit_ratings, movie_info)

Done! (100, 0.00475)


In [25]:
BPR_SGD.similar_items(1)

array([[1, 'Toy Story (1995)', "Animation|Children's|Comedy"],
       [1213, 'GoodFellas (1990)', 'Crime|Drama'],
       [2804, 'Christmas Story, A (1983)', 'Comedy|Drama'],
       [1270, 'Back to the Future (1985)', 'Comedy|Sci-Fi'],
       [1610, 'Hunt for Red October, The (1990)', 'Action|Thriller'],
       [904, 'Rear Window (1954)', 'Mystery|Thriller']], dtype=object)

In [26]:
BPR_SGD.history(4, 10)

array([[2951, 'Fistful of Dollars, A (1964)', 'Action|Western'],
       [1214, 'Alien (1979)', 'Action|Horror|Sci-Fi|Thriller'],
       [1036, 'Die Hard (1988)', 'Action|Thriller'],
       [480, 'Jurassic Park (1993)', 'Action|Adventure|Sci-Fi'],
       [2366, 'King Kong (1933)', 'Action|Adventure|Horror'],
       [1097, 'E.T. the Extra-Terrestrial (1982)',
        "Children's|Drama|Fantasy|Sci-Fi"],
       [3418, 'Thelma & Louise (1991)', 'Action|Drama'],
       [3702, 'Mad Max (1979)', 'Action|Sci-Fi'],
       [3468, 'Hustler, The (1961)', 'Drama'],
       [2692, 'Run Lola Run (Lola rennt) (1998)', 'Action|Crime|Romance']],
      dtype=object)

In [27]:
BPR_SGD.recommend(4, 20)

array([[3753, 'Patriot, The (2000)', 'Action|Drama|War'],
       [2858, 'American Beauty (1999)', 'Comedy|Drama'],
       [1259, 'Stand by Me (1986)', 'Adventure|Comedy|Drama'],
       [589, 'Terminator 2: Judgment Day (1991)',
        'Action|Sci-Fi|Thriller'],
       [356, 'Forrest Gump (1994)', 'Comedy|Romance|War'],
       [457, 'Fugitive, The (1993)', 'Action|Thriller'],
       [3114, 'Toy Story 2 (1999)', "Animation|Children's|Comedy"],
       [318, 'Shawshank Redemption, The (1994)', 'Drama'],
       [608, 'Fargo (1996)', 'Crime|Drama|Thriller'],
       [3176, 'Talented Mr. Ripley, The (1999)',
        'Drama|Mystery|Thriller'],
       [858, 'Godfather, The (1972)', 'Action|Crime|Drama'],
       [3408, 'Erin Brockovich (2000)', 'Drama'],
       [296, 'Pulp Fiction (1994)', 'Crime|Drama'],
       [1193, "One Flew Over the Cuckoo's Nest (1975)", 'Drama'],
       [2599, 'Election (1999)', 'Comedy'],
       [593, 'Silence of the Lambs, The (1991)', 'Drama|Thriller'],
       [1197, '

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных

In [30]:
class WARP:
    def __init__(self, learning_rate = 1e-3, lambda_l2 = 1e-3, 
                 epsilon = 1e-3, latent_size = 16, max_iterations = int(1e7), verbose = int(1e5)):
        self.learning_rate = learning_rate
        self.lambda_l2 = lambda_l2
        self.epsilon = epsilon
        self.latent_size = latent_size
        self.max_iterations = max_iterations
        self.verbose = verbose 
        
    def fit(self, ratings, movie_info):
        
        
        # функция поиска описания фильма по его идентификатору
        self.search = lambda x: movie_info.loc[movie_info.movie_id == x]
        
        # инициализация матриц эмбедингов
        self.W = np.random.uniform(0, 1/np.sqrt(self.latent_size), 
                      size=(ratings.user_id.max(), self.latent_size)) 
        
        self.H = np.random.uniform(0, 1/np.sqrt(self.latent_size), 
                      size=(self.latent_size, ratings.movie_id.max())) 
        set_movies_all = set(np.arange(1, ratings.movie_id.max() + 1))
#         self.biasV = np.random.rand(len(set_movies_all)) * 0.01
        
        ratings_numpy = ratings.to_numpy()
        self.movie_info = movie_info
        self.ratings = ratings
    
        l = ratings.shape[0]
        itx = 0
        # множество всех фильмов
        
        while True:

            # подсчет rmse на кратной итерации
            self.PR = np.array(self.W @ self.H)
            idx_users = ratings_numpy[:, 0] - 1
            idx_movies = ratings_numpy[:, 1] - 1
            rvals = ratings_numpy[:, 2]
            f_norm =  np.linalg.norm(self.PR[idx_users, idx_movies] - rvals) / l
            if f_norm <= self.epsilon:
                break
                    
            for u_pos in tqdm(ratings.user_id.unique(), position=0, leave=False, desc=f'|{itx}. rmse - {f_norm:.10f}'):
                    u_movies = ratings.loc[ratings.user_id == u_pos].movie_id
                    m_pos = u_movies.sample().item()

                    u_pos -= 1
                    m_pos -= 1
                    # выделяем негативные случаи и берем случайный элемент
                    set_movies_pos = set(u_movies)
                    idies_neg = list(set_movies_all - set_movies_pos)
                    for m_neg in idies_neg:
                        m_neg -= 1

                        # считаем расстояние от позитивного отзыва до негативного
                        r_pos = self.W[u_pos, :] @ self.H[:, m_pos]# + self.biasV[m_pos]
                        r_neg = self.W[u_pos, :] @ self.H[:, m_neg]# + self.biasV[m_neg]
                        # обновляем только некорректные оценки
                        if r_neg > r_pos:
                            err = r_pos - r_neg

                            loss = -1.0 / (1 + np.exp(err))
                            self.W[u_pos, :] -= self.learning_rate * (loss * (self.H[:, m_pos].T - self.H[:, m_neg].T)
                                                                  + self.lambda_l2 * self.W[u_pos, :])

                            self.H[:, m_pos] -= self.learning_rate * (loss * self.W[u_pos, :].T + self.lambda_l2 * self.H[:, m_pos])
                            self.H[:, m_neg] -= self.learning_rate * (loss * self.W[u_pos, :].T + self.lambda_l2 * self.H[:, m_neg])


                        
            itx += 1
            if itx >= self.max_iterations:
                break
        
        self.movie_values = self.ratings.movie_id.sort_values().unique()
        print(f'Done! ({itx}, {f_norm:.5f})')
        
    def history(self, idx, count = 5):
        # выгрузка истории просмотренных фильмов по идентификатору пользователя
        idies = self.ratings.loc[self.ratings.user_id == idx].sort_values('rating').movie_id.iloc[:count].values
        histories = pd.concat([self.search(i) for i in idies])
        return histories.to_numpy()
        

    def similar_items(self, idx, count = 5):
        # предсказание схожих по представлениям фильмов
        similar_key = lambda x: np.linalg.norm(self.H[:, x] - self.H[:, idx - 1])
        idx_movies_real = self.ratings.movie_id.sort_values().unique() 
        distances = [[x, np.linalg.norm(similar_key(x - 1))] for x in idx_movies_real]
        distances.sort(key = lambda x: x[1])
        idies = [i[0] for i in distances[:count + 1]]
        similaries = pd.concat([self.search(i) for i in idies])
        return similaries.to_numpy()
    
    def recommend(self, idx, count = 5):
        # рекомендации пользователю 
        user_ratings_all = np.array(self.W @ self.H)[idx - 1]
        user_ratings = set(ratings.loc[ratings.user_id == idx].movie_id)
        sorted_all_idies = np.argsort(user_ratings_all)[::-1] + 1
        idies = [i for i in sorted_all_idies if i not in user_ratings][:count]
        recomendations = pd.concat([self.search(i) for i in idies])
        return recomendations.to_numpy()

In [32]:
WARP_SGD = WARP(learning_rate = 5e-2, lambda_l2 = 1e-3, latent_size = 32, epsilon = 1e-4, max_iterations = 10, verbose = 1)
WARP_SGD.fit(implicit_ratings, movie_info)

Done! (10, 0.00581)


In [33]:
WARP_SGD.similar_items(1)

array([[1, 'Toy Story (1995)', "Animation|Children's|Comedy"],
       [1552, 'Con Air (1997)', 'Action|Adventure|Thriller'],
       [242, 'Farinelli: il castrato (1994)', 'Drama|Musical'],
       [746, 'Force of Evil (1948)', 'Film-Noir'],
       [3754, 'Adventures of Rocky and Bullwinkle, The (2000)',
        "Animation|Children's|Comedy"],
       [1255, 'Bad Taste (1987)', 'Comedy|Horror']], dtype=object)

In [34]:
WARP_SGD.history(4, 10)

array([[2951, 'Fistful of Dollars, A (1964)', 'Action|Western'],
       [1214, 'Alien (1979)', 'Action|Horror|Sci-Fi|Thriller'],
       [1036, 'Die Hard (1988)', 'Action|Thriller'],
       [480, 'Jurassic Park (1993)', 'Action|Adventure|Sci-Fi'],
       [2366, 'King Kong (1933)', 'Action|Adventure|Horror'],
       [1097, 'E.T. the Extra-Terrestrial (1982)',
        "Children's|Drama|Fantasy|Sci-Fi"],
       [3418, 'Thelma & Louise (1991)', 'Action|Drama'],
       [3702, 'Mad Max (1979)', 'Action|Sci-Fi'],
       [3468, 'Hustler, The (1961)', 'Drama'],
       [2692, 'Run Lola Run (Lola rennt) (1998)', 'Action|Crime|Romance']],
      dtype=object)

In [35]:
WARP_SGD.recommend(4, 20)

array([[16, 'Casino (1995)', 'Drama|Thriller'],
       [60, 'Indian in the Cupboard, The (1995)',
        "Adventure|Children's|Fantasy"],
       [969, 'African Queen, The (1951)', 'Action|Adventure|Romance|War'],
       [913, 'Maltese Falcon, The (1941)', 'Film-Noir|Mystery'],
       [3510, 'Frequency (2000)', 'Drama|Thriller'],
       [41, 'Richard III (1995)', 'Drama|War'],
       [1358, 'Sling Blade (1996)', 'Drama|Thriller'],
       [76, 'Screamers (1995)', 'Sci-Fi|Thriller'],
       [39, 'Clueless (1995)', 'Comedy|Romance'],
       [6, 'Heat (1995)', 'Action|Crime|Thriller'],
       [2858, 'American Beauty (1999)', 'Comedy|Drama'],
       [3160, 'Magnolia (1999)', 'Drama'],
       [3148, 'Cider House Rules, The (1999)', 'Drama'],
       [1302, 'Field of Dreams (1989)', 'Drama'],
       [593, 'Silence of the Lambs, The (1991)', 'Drama|Thriller'],
       [3198, 'Papillon (1973)', 'Drama'],
       [3836, "Kelly's Heroes (1970)", 'Action|Comedy|War'],
       [3189, 'My Dog Skip (1999